# CFRM 421 Final Proejct - Terence Chiu

# Introduction

What is the goal of your project?

• What is the data that you are using? What is the original data source if known?

• What does an instance in your data represent (e.g. a person, a transaction, etc.)? How many
instances are there?

• What is the target variable you are trying to predict?

• What are the features used to predict it? Give a few examples of the features.

• Provide any additional relevant information about your data if known (e.g. what is the time
period, what place is it collected from, etc.

# Problem Setup

In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf

In [2]:
prod_weekly = pd.read_csv('domestic_prod.csv', encoding = 'latin1')

In [3]:
net_import_weekly = pd.read_csv('net_import.csv', encoding = 'latin1')

In [11]:
price = pd.read_csv('min_WTI.csv', encoding = 'latin1')

In [5]:
supply_weekly = pd.read_csv('InvestingcomEIA.csv', encoding = 'latin1')
supply_weekly = supply_weekly.iloc[::-1].reset_index(drop=True)
supply_weekly.head()

,Release Date,Time,Actual,Forecast,Previous
0,9-Mar-11,10:30,2.500M,1.100M,-0.400M
1,16-Mar-11,9:30,1.700M,1.800M,2.500M
2,23-Mar-11,9:30,2.100M,2.000M,1.700M
3,30-Mar-11,9:30,2.900M,2.000M,2.100M
4,6-Apr-11,9:30,2.000M,1.300M,2.900M


In [12]:
prod_weekly['ï»¿Date'] = pd.to_datetime(prod_weekly['ï»¿Date'], format='%b %d, %Y', errors='coerce')
net_import_weekly['ï»¿Date'] = pd.to_datetime(net_import_weekly['ï»¿Date'], format='%b %d, %Y', errors='coerce')
supply_weekly['Release Date'] = pd.to_datetime(supply_weekly['Release Date'], format='%d-%b-%y', errors='coerce')
price['Date'] = pd.to_datetime(price['Date'], errors='coerce')
    

start_date = pd.to_datetime('2012-01-01')
end_date = pd.to_datetime('2025-01-01')

prod_weekly = prod_weekly[(prod_weekly['ï»¿Date'] >= start_date) & (prod_weekly['ï»¿Date'] < end_date)]
net_import_weekly = net_import_weekly[(net_import_weekly['ï»¿Date'] >= start_date) & (net_import_weekly['ï»¿Date'] < end_date)]
supply_weekly = supply_weekly[(supply_weekly['Release Date'] >= start_date) & (supply_weekly['Release Date'] < end_date)]

# Filter price data for each report release
filtered_prices = []
for _, report in supply_weekly.iterrows():
    report_time = report['Release Date']
    # Get prices from -60 to +60 minutes around report release
    price_window = price[
        (price['Date'] >= report_time - pd.Timedelta(minutes=60)) &
        (price['Date'] <= report_time + pd.Timedelta(minutes=60))
    ]
    if not price_window.empty:
        filtered_prices.append(price_window)

price = pd.concat(filtered_prices, ignore_index=True)

In [7]:
price.head()

,Datetime_CST,Date,Time,Open,High,Low,Close,Volume,Datetime
0,2012-05-01 00:00:00-05:00,2012-01-05,00:00:00,104.74,104.77,104.74,104.77,26,2012-05-01 00:00:00
1,2012-05-01 00:01:00-05:00,2012-01-05,00:01:00,104.77,104.77,104.76,104.76,6,2012-05-01 00:01:00
2,2012-05-01 00:02:00-05:00,2012-01-05,00:02:00,104.75,104.75,104.75,104.75,2,2012-05-01 00:02:00
3,2012-05-01 00:03:00-05:00,2012-01-05,00:03:00,104.73,104.73,104.73,104.73,1,2012-05-01 00:03:00
4,2012-05-01 00:04:00-05:00,2012-01-05,00:04:00,104.73,104.73,104.73,104.73,1,2012-05-01 00:04:00


In [8]:
prod_weekly.head()

,ï»¿Date,US Weekly Production
1510,2012-01-06,5844.0
1511,2012-01-13,5726.0
1512,2012-01-20,5722.0
1513,2012-01-27,5720.0
1514,2012-02-03,5763.0


In [9]:
net_import_weekly.head()

,ï»¿Date,Weekly Net Import
530,2012-01-06,9871.0
531,2012-01-13,8228.0
532,2012-01-20,8816.0
533,2012-01-27,8843.0
534,2012-02-03,8376.0


In [10]:
supply_weekly.head()

,Release Date,Time,Actual,Forecast,Previous
43,2012-01-05,11:00,2.200M,-1.200M,3.900M
44,2012-01-11,10:30,5.000M,1.000M,2.200M
45,2012-01-19,11:00,-3.400M,2.900M,5.000M
46,2012-01-25,10:30,3.600M,0.900M,-3.400M
47,2012-02-01,10:30,4.200M,3.000M,3.600M


# Algorithms

DNN  
Inputs:
- Weekly Import
- Weekly Production
- Weekly Supply
- Crude Oil Futures Price 1-2 minutes before release  

Outputs (predict): 
- Futures Price 1-2 minutes after release

# Conclusions